In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook

import asdf
from astropy.io import fits
from astropy.wcs import WCS
import ndcube

In [ ]:
test = fits.open("test_data/ib6o23rsq_flt.fits")

In [ ]:
test

In [ ]:
test[0].header

In [ ]:
test[1].header

In [ ]:
#im = plt.imshow(test[1].data[250:350,100:600])
im = plt.imshow(test[1].data)
im.set_clim(0, 100)

In [ ]:
w = WCS(test[1].header)

In [ ]:
ndc = ndcube.NDCube(test[1].data, w)

In [ ]:
ndc

In [ ]:
reference_files = {}
specwcs_filename = "wfc3_ir_specwcs.asdf"
wavelengthrange_filename = "../WFC3_wavelengthrange.asdf"
dist_ref_path = "../WFC3IR_distortion.asdf"

reference_files['distortion'] = dist_ref_path
reference_files['wavelengthrange'] = wavelengthrange_filename
reference_files['specwcs'] = specwcs_filename

In [ ]:
from dispersion_models import DISPXY_Model, DISPXY_Extension
asdf.get_config().add_extension(DISPXY_Extension())

specwcs = asdf.open(reference_files['specwcs']).tree
displ = specwcs['displ']
dispx = specwcs['dispx']
dispy = specwcs['dispy']
invdispl = specwcs['invdispl']
invdispx = specwcs['invdispx']
invdispy = specwcs['invdispy']
orders = specwcs['order']

In [ ]:
invdispx

In [ ]:
# Create grism pipeline

from gwcs import coordinate_frames as cf
from astropy import units as u
from transform_models import WFC3IRForwardGrismDispersion, WFC3IRBackwardGrismDispersion

gdetector = cf.Frame2D(name='grism_detector', 
                       axes_order=(0, 1),
                       unit=(u.pix, u.pix))
det2det = WFC3IRForwardGrismDispersion(orders,
                                        lmodels=displ,
                                        xmodels=invdispx,
                                        ymodels=dispy)
det2det.inverse = WFC3IRBackwardGrismDispersion(orders,
                                              lmodels=invdispl,
                                              xmodels=dispx,
                                              ymodels=dispy)

grism_pipeline = [(gdetector, det2det)]

In [ ]:
from gwcs import WCS

wcsobj = WCS(grism_pipeline)

In [ ]:
print(wcsobj)

In [ ]:
det2det.inverse.evaluate(100,100,2, 1)

In [ ]:
from extraction import extract_2d_spectrum

sliced_data = extract_2d_spectrum(test[1].data, 160, 285, 172, 300)

In [ ]:
sliced_data.shape

In [ ]:
plt.imshow(sliced_data)

In [ ]:
# Checking that the input coordinates I figured out to get that are actually
# where the source is...I worked a bit backwards. 

direct = fits.open("test_data/F140W/ib6o23rtq_flt.fits")

In [ ]:
im2 = plt.imshow(direct[1].data[270:320,140:200])
im2.set_clim(0, 100)